In [10]:
import pandas as pd
import numpy as np
import emd
import pylab as plt
import statsmodels.api as sm
import plotly.express as px
import os
from sklearn import linear_model
np.random.seed(1)

In [2]:
def filter_and_regress(combined_data, country, coal_data = "./Data/Bloomberg/Rotterdam Coal.xlsx", 
        low_pass_percent=0.1, med_pass_percent=0.3, high_pass_percent=0.5, log_adj=20):
        

        combined_data = combined_data.merge(coal_data, on = "Date", how="left")

        combined_data = combined_data[["Last Price", "Price", "Forecast", "Coal Price"]]
        combined_data = combined_data.dropna(axis=0)
        elec_price = combined_data["Price"].to_numpy()
        lng_price = combined_data["Last Price"].to_numpy()
        coal_price  = combined_data["Coal Price"].to_numpy()
        demand = combined_data["Forecast"].to_numpy()
        # plot and transform all of the data for electricity pricing
        #plt.figure()
        #plt.plot(elec_price, "k")

        imf, noise = emd.sift.complete_ensemble_sift(elec_price, ensemble_noise=1)
        # create the pass thresholds based on the input percentages
        low_pass_thresh_elec = int(np.ceil(low_pass_percent * imf.shape[1]))
        med_pass_thresh_elec = int(np.ceil(med_pass_percent * imf.shape[1]))
        high_pass_thresh_elec = int(np.ceil(high_pass_percent * imf.shape[1]))
        #emd.plotting.plot_imfs(imf)

        IP, IF, IA = emd.spectra.frequency_transform(imf, 2156, "hilbert")
        # plot and transform all of the data for LNG prices
        #plt.figure()
        #plt.plot(lng_price, "k")

        lng_imf, lng_noise = emd.sift.complete_ensemble_sift(lng_price, ensemble_noise=1)
        low_pass_thresh_lng = int(np.ceil(low_pass_percent * lng_imf.shape[1]))
        med_pass_thresh_lng = int(np.ceil(med_pass_percent * lng_imf.shape[1]))
        high_pass_thresh_lng = int(np.ceil(high_pass_percent * lng_imf.shape[1]))

        #emd.plotting.plot_imfs(lng_imf)

        IP, IF, IA = emd.spectra.frequency_transform(imf, 2156, "hilbert")

        demand_imf, demand_noise = emd.sift.complete_ensemble_sift(demand, ensemble_noise=1)
        low_pass_thresh_demand = int(np.ceil(low_pass_percent * demand_imf.shape[1]))
        med_pass_thresh_demand = int(np.ceil(med_pass_percent * demand_imf.shape[1]))
        high_pass_thresh_demand = int(np.ceil(high_pass_percent * demand_imf.shape[1]))

        coal_imf, coal_noise = emd.sift.complete_ensemble_sift(coal_price, ensemble_noise=1)
        low_pass_thresh_coal = int(np.ceil(low_pass_percent * coal_imf.shape[1]))
        med_pass_thresh_coal = int(np.ceil(med_pass_percent * coal_imf.shape[1]))
        high_pass_thresh_coal = int(np.ceil(high_pass_percent * coal_imf.shape[1]))
        
        low_pass_elec = imf[:, low_pass_thresh_elec:]
        low_pass_means_elec = np.apply_along_axis(np.mean, 1, low_pass_elec)

        low_pass_lng = lng_imf[:, low_pass_thresh_lng:]
        low_pass_means_lng = np.apply_along_axis(np.mean, 1, low_pass_lng)

        low_pass_demand = demand_imf[:, low_pass_thresh_demand:]
        low_pass_means_demand = np.apply_along_axis(np.mean, 1, low_pass_demand)

        low_pass_coal = coal_imf[:, low_pass_thresh_coal:]
        low_pass_means_coal = np.apply_along_axis(np.mean, 1, low_pass_coal)

        #px.scatter(x=low_pass_means_elec, y=low_pass_means_lng)


        med_pass_elec = imf[:, med_pass_thresh_elec:]
        med_pass_means_elec = np.apply_along_axis(np.mean, 1, med_pass_elec)

        med_pass_lng = lng_imf[:, med_pass_thresh_lng:]
        med_pass_means_lng = np.apply_along_axis(np.mean, 1, med_pass_lng)

        med_pass_demand = demand_imf[:, med_pass_thresh_demand:]
        med_pass_means_demand = np.apply_along_axis(np.mean, 1, med_pass_demand)

        med_pass_coal = coal_imf[:, med_pass_thresh_coal:]
        med_pass_means_coal = np.apply_along_axis(np.mean, 1, med_pass_coal)

        #px.scatter(x=med_pass_means_elec, y=med_pass_means_lng)
        """
        high_pass_elec = imf[:, high_pass_thresh_elec:]
        high_pass_means_elec = np.apply_along_axis(np.mean, 1, high_pass_elec)

        high_pass_lng = lng_imf[:, high_pass_thresh_lng:]
        high_pass_means_lng = np.apply_along_axis(np.mean, 1, high_pass_lng)

        high_pass_demand = lng_imf[:, high_pass_thresh_demand:]
        high_pass_means_demand = np.apply_along_axis(np.mean, 1, high_pass_demand)
        """
        #px.scatter(x=high_pass_means_elec, y=high_pass_means_lng)
        X_low = pd.DataFrame({"LNG": low_pass_means_lng, "Coal": low_pass_means_coal, "Demand": low_pass_means_demand})
        X_low_log = X_low.copy()
        X_low_log["LNG"] = X_low_log["LNG"].apply(lambda x: np.log(x+log_adj))
        X_low_log["Coal"] = X_low_log["Coal"].apply(lambda x: np.log(x+log_adj))
        low_model = linear_model.LinearRegression().fit(X_low_log, np.log(low_pass_means_elec+log_adj))
        print("Low thresh LNG coefficient = {}, Coal Coefficient = {}, Demand Coefficient = {}".format(low_model.coef_[0], low_model.coef_[1], low_model.coef_[2]))

        X_med = pd.DataFrame({"LNG": med_pass_means_lng, "Coal": low_pass_means_coal, "Demand": med_pass_means_demand})
        X_med_log = X_med.copy()
        X_med_log["LNG"] = X_med_log["LNG"].apply(lambda x: np.log(x+log_adj))
        X_med_log["Coal"] = X_med_log["Coal"].apply(lambda x: np.log(x+log_adj))
        med_model = linear_model.LinearRegression().fit(X_med_log, np.log(med_pass_means_elec + log_adj))
        print("Med thresh LNG coefficient = {}, Coal Coefficient = {}, Demand Coefficient = {}".format(med_model.coef_[0], med_model.coef_[1], med_model.coef_[2]))
        """
        X_high = pd.DataFrame({"LNG": high_pass_means_lng, "Demand": high_pass_means_demand})
        X_high_log = X_high.copy()
        X_high_log["LNG"] = X_high_log["LNG"].apply(lambda x: np.log(x + log_adj))
        high_model = linear_model.LinearRegression().fit(X_high_log, np.log(high_pass_means_elec + log_adj))
        print("High thresh LNG coefficient = {},  Demand Coefficent = {}".format(high_model.coef_[0], high_model.coef_[1]))
        """
        return low_model.coef_

In [3]:
#filter_and_regress(pd.read_csv("./Data/Spain/combined_data.csv"), "Spain")
#data = pd.read_csv("./Data/Germany/combined.csv")
#filter_and_regress(data, "Germany")

In [ ]:
def produce_graphs(coefs_pre, coefs_covid, coefs_war, country_name, combined_data, coal_data, log_adj = 25):
        combined_data = combined_data.merge(coal_data, on = "Date", how="left")

        # calculate the columns based on each coefficient
        combined_data = combined_data[["Date", "Last Price", "Price", "Forecast", "Coal Price"]]
        combined_data["Pre Covid Prediction"] = np.exp((np.log(combined_data["Last Price"]+log_adj) * coefs_pre[0]) 
                                                       + (np.log(combined_data["Coal Price"] + log_adj) * coefs_pre[1]) 
                                                       + (combined_data["Forecast"] * coefs_pre[2]))

        combined_data["During Covid Prediction"] = np.exp((np.log(combined_data["Last Price"]+log_adj) * coefs_covid[0]) 
                                                       + (np.log(combined_data["Coal Price"] + log_adj) * coefs_covid[1]) 
                                                       + (combined_data["Forecast"] * coefs_covid[2]))
        
        fig = px.line(combined_data, x = combined_data["Date"])



In [4]:
# run regressions based on the timescales of COVID and war in Ukraine
def timeperiod_differences(combined_data_path, country_name, coal_path = "./Data/Bloomberg/Rotterdam Coal.xlsx",
                            log_adj=20):
        # these serve as best guesses, change at will
        COVID_START = "2020-03-01"
        COVID_END = "2021-06-20" # 50% of Europe was vaccinated by this date: 
        # https://www.bbc.com/news/explainers-52380823
        WAR_START = "2022-02-01"

        coal_data = pd.read_excel(coal_path)
        coal_data["Date"] = pd.to_datetime(coal_data["Date"])
        coal_data["Coal Price"] = coal_data["Last Price"]
        coal_data = coal_data[["Date", "Coal Price"]]

        # read in the data from the combined dataset
        data = pd.read_csv(combined_data_path)
        data["Date"] = pd.to_datetime(data["Date"])
        pre_covid = data[data["Date"] < COVID_START]

        covid = data[data["Date"] > COVID_START]
        covid = covid[covid["Date"] < COVID_END]

        war = data[data["Date"] > WAR_START]

        # run the regressions on the given datasets
        print("Pre-COVID in {}".format(country_name))
        filter_and_regress(pre_covid, country_name, coal_data, log_adj=log_adj)

        print("COVID Era in {}".format(country_name))
        filter_and_regress(covid, country_name, coal_data, log_adj=log_adj)

        print("War in Ukraine Era in {}".format(country_name))
        filter_and_regress(war, country_name, coal_data, log_adj=log_adj)





In [5]:
timeperiod_differences("./Data/Spain/combined_data.csv", "Spain")

Pre-COVID in Spain
Low thresh LNG coefficient = 0.5857487621381581, Coal Coefficient = 0.18239740454597186, Demand Coefficient = 4.8240494594453676e-05
Med thresh LNG coefficient = 0.5726458550194345, Coal Coefficient = 0.22766980149121152, Demand Coefficient = 4.4040392168764875e-05
COVID Era in Spain
Low thresh LNG coefficient = 1.9522273640193555, Coal Coefficient = 0.2415844441287966, Demand Coefficient = 5.451028754924203e-05
Med thresh LNG coefficient = 1.846089272643852, Coal Coefficient = 0.3241274857181896, Demand Coefficient = 5.2688305945625125e-05
War in Ukraine Era in Spain
Low thresh LNG coefficient = 0.15116118446938928, Coal Coefficient = 0.027383726225790894, Demand Coefficient = 7.823311476709213e-05
Med thresh LNG coefficient = 0.16524964643106407, Coal Coefficient = 0.029702191322380455, Demand Coefficient = 6.43680415261666e-05


In [6]:
timeperiod_differences("./Data/Netherlands/combined_data.csv", "Netherlands")

Pre-COVID in Netherlands
Low thresh LNG coefficient = 0.4629615292114902, Coal Coefficient = 0.10974110434430268, Demand Coefficient = 8.735326546827071e-05
Med thresh LNG coefficient = 0.455233345054619, Coal Coefficient = 0.13420897349075644, Demand Coefficient = 7.865397987416589e-05
COVID Era in Netherlands
Low thresh LNG coefficient = 1.7498582219427044, Coal Coefficient = 0.2526381581427471, Demand Coefficient = 0.00015502375732029394
Med thresh LNG coefficient = 1.712391706016884, Coal Coefficient = 0.32361701541253485, Demand Coefficient = 0.00013915941078768146
War in Ukraine Era in Netherlands
Low thresh LNG coefficient = 0.6073139018862164, Coal Coefficient = 0.11697015334935729, Demand Coefficient = 0.0003591021555504753
Med thresh LNG coefficient = 0.5159735682554603, Coal Coefficient = 0.1336356058124748, Demand Coefficient = 0.0002702813415114136


In [7]:
timeperiod_differences("./Data/Germany/combined_data.csv", "Germany", log_adj=25)

Pre-COVID in Germany
Low thresh LNG coefficient = 0.7531126366502925, Coal Coefficient = -0.06006145847715832, Demand Coefficient = 1.9456916323858098e-05
Med thresh LNG coefficient = 0.7075531310662562, Coal Coefficient = -0.06800574106344158, Demand Coefficient = 1.831611373401265e-05
COVID Era in Germany
Low thresh LNG coefficient = 1.6559899445914863, Coal Coefficient = -0.006968466408201935, Demand Coefficient = 2.589488472967607e-05
Med thresh LNG coefficient = 1.5771802187103297, Coal Coefficient = 0.04879473470598535, Demand Coefficient = 2.5227583800011e-05
War in Ukraine Era in Germany
Low thresh LNG coefficient = 0.9633830024895748, Coal Coefficient = 0.1310632732764151, Demand Coefficient = 5.553805726274397e-05
Med thresh LNG coefficient = 0.9167253526048825, Coal Coefficient = 0.15991580045736586, Demand Coefficient = 5.118115219041042e-05


In [8]:
timeperiod_differences("./Data/France/combined_data.csv", "France", log_adj=26)

Pre-COVID in France
Low thresh LNG coefficient = 0.5487140259765162, Coal Coefficient = 0.18177118164301567, Demand Coefficient = 2.8097283434197484e-05
Med thresh LNG coefficient = 0.5667245063853805, Coal Coefficient = 0.22322692583502388, Demand Coefficient = 2.6874740560983712e-05
COVID Era in France
Low thresh LNG coefficient = 1.4828437915650174, Coal Coefficient = 0.8970037761389087, Demand Coefficient = 2.622826058018557e-05
Med thresh LNG coefficient = 1.411932493657374, Coal Coefficient = 1.1456352065189146, Demand Coefficient = 2.4225400100164052e-05
War in Ukraine Era in France
Low thresh LNG coefficient = 0.6949480514218042, Coal Coefficient = 0.10164385772143439, Demand Coefficient = 4.166870366979769e-05
Med thresh LNG coefficient = 0.6955890922851112, Coal Coefficient = 0.12008529928264351, Demand Coefficient = 3.5860851091848445e-05


In [9]:
timeperiod_differences("./Data/Austria/combined_data.csv", "Austria")

Pre-COVID in Austria
Low thresh LNG coefficient = 0.25525286030850086, Coal Coefficient = 0.04572806444104268, Demand Coefficient = -8.662411717583196e-07
Med thresh LNG coefficient = 0.24884337145007415, Coal Coefficient = 0.0660603272828229, Demand Coefficient = -1.1020175089831996e-06
COVID Era in Austria
Low thresh LNG coefficient = 1.333699386342992, Coal Coefficient = 0.46744853117351887, Demand Coefficient = 0.00027472854199234176
Med thresh LNG coefficient = 1.193142304974083, Coal Coefficient = 0.5991790955953099, Demand Coefficient = 0.00024759398735241245
War in Ukraine Era in Austria
Low thresh LNG coefficient = 1.0423853035392345, Coal Coefficient = 0.10184811656238615, Demand Coefficient = 0.0005148987952815764
Med thresh LNG coefficient = 0.9780851810673976, Coal Coefficient = 0.1246997418446105, Demand Coefficient = 0.00039703437611784964
